In [2]:
!pip uninstall -y ipython-sql
!pip install jupysql


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install trino sqlalchemy-trino

  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0━━━━━━━━━━━━━━━━━━━━ 1/5 [requests]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [sqlalchemy-trino][trino]

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install --upgrade "grpcio>=1.48.1" google-auth


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [113]:
%load_ext sql
    
# เชื่อมต่อแค่ระดับ catalog (iceberg)
%sql trino://admin@trino-coordinator:8080/iceberg

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [104]:
%%sql
CREATE TABLE IF NOT EXISTS test_db.app_logs2 (
    log_id BIGINT,
    log_level VARCHAR,
    message VARCHAR,
    event_time TIMESTAMP(6)
)
WITH (
    partitioning = ARRAY['day(event_time)'],
    format = 'PARQUET'
)

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

++
||
++
++

In [102]:
%%sql
INSERT INTO iceberg.test_db.app_logs VALUES 
(101, 'INFO', 'User login success', TIMESTAMP '2026-01-31 23:50:00')

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

RuntimeError: (trino.exceptions.TrinoQueryError) TrinoQueryError(type=INTERNAL_ERROR, name=REMOTE_TASK_ERROR, message="Unexpected response from http://172.30.0.11:8080/v1/task/20260201_091550_00138_2dmtv.1.0.0?summarize", query_id=20260201_091550_00138_2dmtv)
[SQL: INSERT INTO iceberg.test_db.app_logs VALUES
(101, 'INFO', 'User login success', TIMESTAMP '2026-01-31 23:50:00')]
(Background on this error at: https://sqlalche.me/e/20/dbapi)


In [105]:
%%sql
SELECT * FROM test_db."app_logs2$partitions"

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

record_count,file_count,total_size,data
30,3,2927,"(log_id: (min: 101, max: 10211, null_count: 0, nan_count: None), log_level: (min: 'DEBUG', max: 'WARN', null_count: 0, nan_count: None), message: (min: 'Backup finished', max: 'User logout', null_count: 0, nan_count: None), event_time: (min: datetime.datetime(2026, 1, 25, 8, 0), max: datetime.datetime(2026, 2, 2, 14, 0), null_count: 0, nan_count: None))"


In [106]:
%%sql
SELECT 
    CAST(event_time AS DATE) as log_date, 
    count(*) as row_count 
FROM iceberg.test_db.app_logs 
GROUP BY 1 
ORDER BY 1

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

log_date,row_count
2026-01-28,6
2026-01-29,6
2026-01-30,4
2026-01-31,5
2026-02-01,9
2026-02-02,2


In [107]:
%%sql
SELECT partition, record_count, file_path 
FROM iceberg.test_db.app_logs$files

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(trino.exceptions.TrinoUserError) TrinoUserError(type=USER_ERROR, name=SYNTAX_ERROR, message="line 2:30: mismatched input '$'. Expecting: ',', '.', 'AS', 'CROSS', 'EXCEPT', 'FETCH', 'FOR', 'FULL', 'GROUP', 'HAVING', 'INNER', 'INTERSECT', 'JOIN', 'LEFT', 'LIMIT', 'MATCH_RECOGNIZE', 'NATURAL', 'OFFSET', 'ORDER', 'RIGHT', 'TABLESAMPLE', 'UNION', 'WHERE', 'WINDOW', <EOF>, <identifier>", query_id=20260201_091600_00145_2dmtv)
[SQL: SELECT partition, record_count, file_path
FROM iceberg.test_db.app_logs$files]
(Background on this error at: https://sqlalche.me/e/20/f405)



In [108]:
%%sql
SELECT content, file_path, record_count 
FROM test_db."app_logs2$files"

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

content,file_path,record_count
0,s3a://warehouse//app_logs2-1f06037e067949f0bb21d483c15b6766/data/20260201_090637_00093_2dmtv-aa5e9c1a-fc04-40e5-ae58-b450cd51c864.parquet,10
0,s3a://warehouse//app_logs2-1f06037e067949f0bb21d483c15b6766/data/20260201_085844_00062_2dmtv-84e3b663-1065-4e6a-8c2a-a0c0bbdc79b0.parquet,10
0,s3a://warehouse//app_logs2-1f06037e067949f0bb21d483c15b6766/data/20260201_090841_00103_2dmtv-22ac2f67-2637-42c5-8297-d350e8fd378c.parquet,10


In [109]:
%%sql
SELECT * FROM test_db."app_logs2$partitions"

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

record_count,file_count,total_size,data
30,3,2927,"(log_id: (min: 101, max: 10211, null_count: 0, nan_count: None), log_level: (min: 'DEBUG', max: 'WARN', null_count: 0, nan_count: None), message: (min: 'Backup finished', max: 'User logout', null_count: 0, nan_count: None), event_time: (min: datetime.datetime(2026, 1, 25, 8, 0), max: datetime.datetime(2026, 2, 2, 14, 0), null_count: 0, nan_count: None))"


In [114]:
%%sql

INSERT INTO test_db.app_logs2 (log_id, log_level, message, event_time) VALUES
(1012, 'INFO',  'User login success', TIMESTAMP '2025-01-25 08:00:00'),
(1023, 'INFO',  'User login success', TIMESTAMP '2025-01-26 09:00:00'),
(1032, 'WARN',  'High CPU usage',     TIMESTAMP '2025-01-26 10:30:00'),
(1044, 'ERROR', 'Service crash',      TIMESTAMP '2025-01-27 12:00:00'),
(1054, 'INFO',  'Service restarted',  TIMESTAMP '2025-01-27 12:05:00'),
(1065, 'INFO',  'User logout',        TIMESTAMP '2025-01-28 15:00:00'),
(1076, 'DEBUG', 'Internal trace',     TIMESTAMP '2025-01-29 01:00:00'),
(1078, 'INFO',  'Backup started',     TIMESTAMP '2025-01-30 22:00:00'),
(1089, 'INFO',  'Backup finished',    TIMESTAMP '2025-01-30 23:30:00'),
(1180, 'ERROR', 'Database locked',    TIMESTAMP '2025-02-02 14:00:00')


Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

10 rows affected.

rows
10


In [122]:
%%sql
SELECT 
    -- แปลง event_time เป็นแค่วันที่ เพื่อดูยอดรวมต่อวัน
    CAST(event_time AS DATE) as log_date, 
    COUNT(*) as total_rows
FROM test_db.app_logs2
GROUP BY CAST(event_time AS DATE)
ORDER BY log_date

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

log_date,total_rows
2025-01-25,1
2025-01-26,2
2025-01-27,2
2025-01-28,1
2025-01-29,1
2025-01-30,2
2025-02-02,1
2026-01-25,4
2026-01-26,8
2026-01-27,8


In [116]:
%%sql

SELECT * FROM test_db."app_logs2$partitions"

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

record_count,file_count,total_size,data
50,5,4881,"(log_id: (min: 101, max: 10211, null_count: 0, nan_count: None), log_level: (min: 'DEBUG', max: 'WARN', null_count: 0, nan_count: None), message: (min: 'Backup finished', max: 'User logout', null_count: 0, nan_count: None), event_time: (min: datetime.datetime(2025, 1, 25, 8, 0), max: datetime.datetime(2026, 2, 2, 14, 0), null_count: 0, nan_count: None))"


In [117]:
%%sql
EXPLAIN SELECT * FROM test_db.app_logs2 
WHERE event_time >= TIMESTAMP '2026-02-01 00:00:00'

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

Query Plan
"Fragment 0 [SOURCE] Output layout: [log_id, log_level, message, event_time] Output partitioning: SINGLE [] Output[columnNames = [log_id, log_level, message, event_time]] │ Layout: [log_id:bigint, log_level:varchar, message:varchar, event_time:timestamp(6)] │ Estimates: {rows: 0 (12B), cpu: 0, memory: 0B, network: 0B} └─ ScanFilter[table = iceberg:test_db.app_logs2$data@7502507685830457803, filterPredicate = (""event_time"" >= TIMESTAMP '2026-02-01 00:00:00.000000')] Layout: [log_id:bigint, log_level:varchar, message:varchar, event_time:timestamp(6)] Estimates: {rows: 50 (2.79kB), cpu: 2.79k, memory: 0B, network: 0B}/{rows: 0 (12B), cpu: 2.79k, memory: 0B, network: 0B} log_id := 1:log_id:bigint log_level := 2:log_level:varchar message := 3:message:varchar event_time := 4:event_time:timestamp(6)"


In [120]:
%%sql
EXPLAIN ANALYZE SELECT * FROM test_db.app_logs2 
WHERE event_time >= TIMESTAMP '2026-01-03 00:00:00'

Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

Query Plan
"Fragment 1 [SOURCE] CPU: 22.40ms, Scheduled: 42.69ms, Blocked 0.00ns (Input: 0.00ns, Output: 0.00ns), Input: 40 rows (1.84kB); per task: avg.: 40.00 std.dev.: 0.00, Output: 40 rows (1.84kB) Output layout: [log_id, log_level, message, event_time] Output partitioning: SINGLE [] ScanFilter[table = iceberg:test_db.app_logs2$data@7502507685830457803, filterPredicate = (""event_time"" >= TIMESTAMP '2026-01-03 00:00:00.000000')] Layout: [log_id:bigint, log_level:varchar, message:varchar, event_time:timestamp(6)] Estimates: {rows: 50 (2.79kB), cpu: 2.79k, memory: 0B, network: 0B}/{rows: 4 (234B), cpu: 2.79k, memory: 0B, network: 0B} CPU: 22.00ms (100.00%), Scheduled: 42.00ms (100.00%), Blocked: 0.00ns (?%), Output: 40 rows (1.84kB) Input avg.: 10.00 rows, Input std.dev.: 0.00% log_id := 1:log_id:bigint log_level := 2:log_level:varchar message := 3:message:varchar event_time := 4:event_time:timestamp(6) Input: 40 rows (1.84kB), Filtered: 0.00%"


Running query in 'trino://admin@trino-coordinator:8080/iceberg/test_db'

RuntimeError: (trino.exceptions.TrinoQueryError) TrinoQueryError(type=INTERNAL_ERROR, name=REMOTE_TASK_ERROR, message="Unexpected response from http://172.30.0.11:8080/v1/task/20260201_091725_00169_2dmtv.1.0.0?summarize", query_id=20260201_091725_00169_2dmtv)
[SQL: ALTER TABLE test_db.app_logs2 EXECUTE optimize]
(Background on this error at: https://sqlalche.me/e/20/dbapi)
